<a href="https://colab.research.google.com/github/lbrandoli/ProgramacionConcurrente/blob/main/TP_INTEGRADOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SERVIDOR

In [ ]:
import threading
import socket
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# Clave de cifrado
key = b'clave_secreta'

# Función para cifrar un mensaje
def encrypt(message):
    cipher = AES.new(key, AES.MODE_EAX)
    nonce = cipher.nonce
    ciphertext, tag = cipher.encrypt_and_digest(message.encode('utf-8'))
    return nonce + ciphertext + tag

# Función para descifrar un mensaje
def decrypt(ciphertext):
    nonce = ciphertext[:16]
    tag = ciphertext[-16:]
    ciphertext = ciphertext[16:-16]
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    plaintext = cipher.decrypt(ciphertext)
    try:
        cipher.verify(tag)
        return plaintext.decode('utf-8')
    except:
        return None

# Función para manejar la conexión con un cliente
def handle_client(conn, addr):
    print(f'Conexión establecida con {addr}')
    while True:
        # Recibir mensaje cifrado del cliente
        ciphertext = conn.recv(1024)
        if not ciphertext:
            break
        # Descifrar mensaje
        message = decrypt(ciphertext)
        if message:
            print(f'Mensaje recibido de {addr}: {message}')
            # Cifrar respuesta
            response = encrypt(f'Tu mensaje "{message}" ha sido recibido.')
            # Enviar respuesta cifrada al cliente
            conn.sendall(response)
    print(f'Conexión cerrada con {addr}')
    conn.close()

# Función para manejar las conexiones entrantes
def handle_connections():
    # Crear socket
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(('localhost', 12345))
    s.listen()
    print('Servidor iniciado en localhost:1234')
    while True:
        # Aceptar conexión entrante
        conn, addr = s.accept()
        # Crear hilo para manejar la conexión
        t = threading.Thread(target=handle_client, args=(conn, addr))
        t.start()

# Iniciar servidor
handle_connections()

CLIENTE

In [ ]:
import socket
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# Generar una nueva clave de cifrado AES de 16 bytes
#key = get_random_bytes(16)

# Clave de cifrado
key = b'clave_secreta'

# Función para cifrar un mensaje
def encrypt(message):
    cipher = AES.new(key, AES.MODE_EAX)
    nonce = cipher.nonce
    ciphertext, tag = cipher.encrypt_and_digest(message.encode('utf-8'))
    return nonce + ciphertext + tag

# Función para descifrar un mensaje
def decrypt(ciphertext):
    nonce = ciphertext[:16]
    tag = ciphertext[-16:]
    ciphertext = ciphertext[16:-16]
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    plaintext = cipher.decrypt(ciphertext)
    try:
        cipher.verify(tag)
        return plaintext.decode('utf-8')
    except:
        return None

# Crear socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(('localhost', 12345))

while True:
    # Leer mensaje desde la consola
    message = input('Mensaje: ')
    # Cifrar mensaje
    ciphertext = encrypt(message)
    # Enviar mensaje cifrado al servidor
    s.sendall(ciphertext)
    # Recibir respuesta cifrada del servidor
    ciphertext = s.recv(1024)
    # Descifrar respuesta
    response = decrypt(ciphertext)
    if response:
        print(f'Respuesta: {response}')
    else:
        print('Error al descifrar la respuesta')
        break

# Cerrar conexión
s.close()

MAIN

In [ ]:
import subprocess
import time

def run():
    # Iniciar el servidor como un proceso padre
    servidor_proceso = subprocess.Popen(["python", "servidor.py"])

    # Esperar un breve momento antes de iniciar los clientes
    time.sleep(2)

    # Iniciar dos clientes como procesos hijos
    cliente_proceso_1 = subprocess.Popen(["python", "cliente.py"])
    cliente_proceso_2 = subprocess.Popen(["python", "cliente.py"])

    # Esperar a que los procesos hijos terminen
    cliente_proceso_1.wait()
    cliente_proceso_2.wait()

    # Terminar el servidor después de que los clientes hayan terminado
    servidor_proceso.terminate()


if __name__ == "__main__":
    run()